<a href="https://colab.research.google.com/github/kboroz/DeepFinance/blob/main/ModelParameterVasicek.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import tensorflow as tf

from keras.models import Sequential
from keras.layers import Input, Dense, Conv2D, Concatenate, Dropout, Subtract, \
                        Flatten, MaxPooling2D, Multiply, Lambda, Add, Dot
from keras.backend import constant
from keras import optimizers

from tensorflow.keras.layers import Layer, InputSpec
from keras.models import Model
from keras.layers import Input
from keras import initializers
from keras.constraints import max_norm
import keras.backend as K

import matplotlib.pyplot as plt

In [3]:
#Define Bond-Pricing formula

def bondprice(r,kappa,theta,sigma,T):
    B=(1-np.exp(-kappa*T))/kappa
    A=np.exp((theta-sigma**2/(2*kappa**2))*( B-T) - sigma**2/(4 *kappa)* B**2)
    P=A*np.exp(-r*B)
    return P    

In [4]:
#Define Parameters for simluation study

Ktrain = 10**4

r=np.random.uniform(-0.00,0.03,size=(Ktrain,1))
kappa=np.random.uniform(0.3,1.3,size=(Ktrain,1))
theta=np.random.uniform(-0.00,0.03,size=(Ktrain,1))
sigma=np.random.uniform(0.1,0.4,size=(Ktrain,1))

In [5]:
#Calculate Bond-Prices

prices=bondprice(r,kappa,theta,sigma,[1/12,2/12,3/12,4/12,5/12,6/12,7/12, 8/12, 9/12, 10/12, 11/12, 1, 2,3,4,5,10])

In [6]:
#Data Check

print(prices)

[[0.99782887 0.99577153 0.99383848 ... 1.01290054 1.03414716 1.15925649]
 [0.99796597 0.99603921 0.99426501 ... 1.3010651  1.53765814 4.31086066]
 [0.9986588  0.99733378 0.99606045 ... 1.06573705 1.10913254 1.37523239]
 ...
 [0.99913098 0.99833073 0.99764102 ... 1.1560333  1.23841993 1.78285702]
 [0.99888894 0.99777814 0.99669673 ... 1.00131864 1.00829757 1.04501978]
 [0.99832314 0.99669324 0.99515054 ... 1.22300023 1.399613   3.26846648]]


In [7]:
#Define Data aka Prices and Parameters

xtrain=np.concatenate((prices, r), axis=1)
ytrain=np.concatenate((kappa,theta,sigma), axis=1)

In [8]:
#Define NN

model = Sequential()

layer1=Dense(64, input_shape=(18,),activation=tf.nn.relu, use_bias=True, kernel_initializer='random_uniform')
layer1.trainable=True
model.add(layer1)
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(3,activation='linear')) 

In [9]:
#Training

model.compile(optimizer='adam', 
              loss='mean_squared_error')

model.fit(xtrain,ytrain, epochs=100)

Epoch 1/100
313/313 [==============================] - 1s 2ms/step - loss: 0.0263
Epoch 2/100
313/313 [==============================] - 1s 2ms/step - loss: 0.0173
Epoch 3/100
313/313 [==============================] - 1s 2ms/step - loss: 0.0168
Epoch 4/100
313/313 [==============================] - 1s 2ms/step - loss: 0.0166
Epoch 5/100
313/313 [==============================] - 1s 2ms/step - loss: 0.0162
Epoch 6/100
313/313 [==============================] - 1s 2ms/step - loss: 0.0149
Epoch 7/100
313/313 [==============================] - 1s 2ms/step - loss: 0.0133
Epoch 8/100
313/313 [==============================] - 1s 2ms/step - loss: 0.0112
Epoch 9/100
313/313 [==============================] - 1s 2ms/step - loss: 0.0097
Epoch 10/100
313/313 [==============================] - 1s 2ms/step - loss: 0.0083
Epoch 11/100
313/313 [==============================] - 1s 2ms/step - loss: 0.0072
Epoch 12/100
313/313 [==============================] - 1s 2ms/step - loss: 0.0065
Epoch 13/100


In [11]:
#Testing data

Ktest = 1

rtest=np.random.uniform(-0.00,0.03,size=(Ktest,1))
kappatest=np.random.uniform(0.3,1.3,size=(Ktest,1))
thetatest=np.random.uniform(-0.00,0.03,size=(Ktest,1))
sigmatest=np.random.uniform(0.1,0.4,size=(Ktest,1))

pricestest=bondprice(rtest,kappatest,thetatest,sigmatest,[1/12,2/12,3/12,4/12,5/12,6/12,7/12, 8/12, 9/12, 10/12, 11/12, 1, 2,3,4,5,10])

xtest=np.concatenate((pricestest, rtest), axis=1)
ytest=np.concatenate((kappatest,thetatest,sigmatest), axis=1)

In [12]:
#Compare model parameters

print(model.predict(xtest))
print(ytest)

[[0.47350538 0.01625757 0.21983261]]
[[0.49656094 0.0171911  0.22500875]]
